In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import numpy as np
import torch
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, random_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import KFold

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
folders = ["test", "train", "valid"]
inner_folders = ["fake", "real"]

total_fake = 0
total_real = 0

for i in range(len(folders)):
  for j in range(len(inner_folders)):
    x = os.listdir(f"/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake/{folders[i]}/{inner_folders[j]}")
    print(f"Length of {inner_folders[j]} images in {folders[i]} : {len(x)}")
    if(inner_folders[j] == "real"):
      total_real += len(x)
    else:
      total_fake += len(x)

print(f"Total Number of Real images in dataset: {total_real}")

print(f"Total Number of Fake images in dataset: {total_fake}")

Length of fake images in test : 10000
Length of real images in test : 10000
Length of fake images in train : 50000
Length of real images in train : 50000
Length of fake images in valid : 10000
Length of real images in valid : 10000
Total Number of Real images in dataset: 70000
Total Number of Fake images in dataset: 70000


In [4]:
data_dir = "/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake"

In [5]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Grayscale(num_output_channels=3),
        transforms.Resize((64, 64)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Grayscale(num_output_channels=3),
        transforms.Resize((64, 64)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Grayscale(num_output_channels=3),
        transforms.Resize((64, 64)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [6]:
data_dirs = {
    'train': os.path.join(data_dir, 'train'),
    'val': os.path.join(data_dir, 'valid'),
    'test': os.path.join(data_dir, 'test')
}

In [7]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

train_dataset = ImageFolder(root=data_dirs['train'], transform=data_transforms['train'])
val_dataset = ImageFolder(root=data_dirs['val'], transform=data_transforms['val'])
test_dataset = ImageFolder(root=data_dirs['test'], transform=data_transforms['test'])

In [8]:
import torch
from torch.utils.data import DataLoader, Subset, ConcatDataset

# Total number of samples to select
train_size = 60000
val_size = 12000
test_size = 12000

# Create indices for subsets
train_indices = torch.arange(train_size)
val_indices = torch.arange(val_size)
test_indices = torch.arange(test_size)

# Create subsets
train_subset = Subset(train_dataset, train_indices)
val_subset = Subset(val_dataset, val_indices)
test_subset = Subset(test_dataset, test_indices)

# Create DataLoaders for subsets
train_loader = DataLoader(train_subset, batch_size=200, shuffle=True, pin_memory=True, num_workers=2)
val_loader = DataLoader(val_subset, batch_size=200, shuffle=False, pin_memory=True, num_workers=2)
test_loader = DataLoader(test_subset, batch_size=1, shuffle=False, pin_memory=True, num_workers=2)

# Combine train and validation subsets
combined_dataset = ConcatDataset([train_subset, val_subset])
combined_loader = DataLoader(combined_dataset, batch_size=200, shuffle=True, pin_memory=True, num_workers=2)

print(f"Train set: {len(train_subset)}, Val set: {len(val_subset)}, Test set: {len(test_subset)}")

Train set: 60000, Val set: 12000, Test set: 12000


In [15]:
class CustomMobileNet(nn.Module):
    def __init__(self):
        super(CustomMobileNet, self).__init__()
        mobilenet = models.mobilenet_v2(weights='IMAGENET1K_V1')
        self.features = mobilenet.features
        self.classifier = nn.Sequential(
            nn.Linear(mobilenet.last_channel, 4096),  # Adjust for 64x64 images
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 1)  # Adjust num_classes
        )
        # Freeze all parameters initially
        for param in self.features.parameters():
            param.requires_grad = False
    
        #self.freeze_features()
    

    def forward(self, x):
        x = self.features(x)
        x = x.mean([2, 3])  # Global average pooling
        x = self.classifier(x)
        return x

    

    def unfreeze_last_layers(self, num_layers):
        layers = list(self.features.children())[-num_layers:]
        for layer in layers:
            for param in layer.parameters():
                param.requires_grad = True

In [10]:
from torch.utils.data import DataLoader, ConcatDataset, Subset

def train_model(model, combined_dataset, criterion, optimizer, num_epochs=10, k=3):
    kfold = KFold(n_splits=k, shuffle=True, random_state=42)
    
    
    fold_metrics = {
        "train_losses": [],
        "valid_losses": [],
        "train_accuracies": [],
        "valid_accuracies": [],
        "precisions": [],
        "recalls": [],
        "best_accuracy": []
    }

    for fold, (train_idx, valid_idx) in enumerate(kfold.split(combined_dataset)):
        print(f"\nFold {fold+1}/{k}")

        
        train_subset = Subset(combined_dataset, train_idx)
        valid_subset = Subset(combined_dataset, valid_idx)

        train_loader = DataLoader(train_subset, batch_size=200, shuffle=True)
        valid_loader = DataLoader(valid_subset, batch_size=200, shuffle=False)

        
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {device}")
    
        
        model = model.to(device)
        criterion = criterion.to(device)

        best_val_accuracy = 0.0
        fold_precision = 0
        fold_recall = 0

        for epoch in range(num_epochs):
            model.train()
            running_loss = 0.0
            correct = 0
            total = 0

            
            with tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch", total=len(train_loader)) as tepoch:
                for inputs, labels in tepoch:
                    inputs, labels = inputs.to(device), labels.to(device)
                    labels = labels.float()  # Ensure labels are float for BCEWithLogitsLoss

                    optimizer.zero_grad()

                    # Forward pass
                    outputs = model(inputs)
                    loss = criterion(outputs.squeeze(), labels) 

                    # Backward pass and optimization
                    loss.backward()
                    optimizer.step()

                    # Track training statistics
                    running_loss += loss.item()
                    preds = torch.round(torch.sigmoid(outputs))
                    correct += (preds.squeeze() == labels).sum().item()
                    total += labels.size(0)

                    # Update progress bar
                    tepoch.set_postfix(loss=running_loss / (tepoch.n + 1), accuracy=correct / total * 100)

            
            train_accuracy = correct / total * 100
            fold_metrics["train_losses"].append(running_loss / len(train_loader))
            fold_metrics["train_accuracies"].append(train_accuracy)

            print(f"Train Loss: {running_loss / len(train_loader):.4f}, Train Accuracy: {train_accuracy:.2f}%") 

            # Validation loop with tqdm progress bar
            model.eval()
            correct = 0
            total = 0
            running_valid_loss = 0.0
            all_preds = []
            all_labels = []
            with tqdm(valid_loader, desc=f"Validation Epoch {epoch+1}/{num_epochs}", unit="batch", total=len(valid_loader)) as vepoch:
                with torch.no_grad():
                    for inputs, labels in vepoch:
                        inputs, labels = inputs.to(device), labels.to(device)
                        labels = labels.float()  # Ensure labels are float for BCEWithLogitsLoss
                        outputs = model(inputs)
                        loss = criterion(outputs.squeeze(), labels)

                        running_valid_loss += loss.item()

                        preds = torch.round(torch.sigmoid(outputs))
                        correct += (preds.squeeze() == labels).sum().item()
                        total += labels.size(0)

                        all_preds.extend(preds.squeeze().cpu().numpy())
                        all_labels.extend(labels.cpu().numpy())

                        # Update progress bar
                        vepoch.set_postfix(loss=running_valid_loss / (vepoch.n + 1), accuracy=correct / total * 100)

                val_accuracy = correct / total * 100
                fold_metrics["valid_losses"].append(running_valid_loss / len(valid_loader))
                fold_metrics["valid_accuracies"].append(val_accuracy)

            
            precision = precision_score(all_labels, all_preds)
            recall = recall_score(all_labels, all_preds)
            fold_precision += precision
            fold_recall += recall

            print(f"Validation Accuracy: {val_accuracy:.2f}%")
            print(f"Precision: {precision:.2f}, Recall: {recall:.2f}")

            # Save the best model based on validation accuracy
            if val_accuracy > best_val_accuracy:
                best_val_accuracy = val_accuracy
                torch.save(model.state_dict(), f"best_model_fold_{fold+1}.pth")

        
        fold_metrics["best_accuracy"].append(best_val_accuracy)
        fold_metrics["precisions"].append(fold_precision / num_epochs)
        fold_metrics["recalls"].append(fold_recall / num_epochs)

    
    avg_train_loss = sum(fold_metrics["train_losses"]) / len(fold_metrics["train_losses"])
    avg_valid_loss = sum(fold_metrics["valid_losses"]) / len(fold_metrics["valid_losses"])
    avg_train_accuracy = sum(fold_metrics["train_accuracies"]) / len(fold_metrics["train_accuracies"])
    avg_valid_accuracy = sum(fold_metrics["valid_accuracies"]) / len(fold_metrics["valid_accuracies"])
    avg_precision = sum(fold_metrics["precisions"]) / len(fold_metrics["precisions"])
    avg_recall = sum(fold_metrics["recalls"]) / len(fold_metrics["recalls"])
    avg_best_accuracy = sum(fold_metrics["best_accuracy"]) / len(fold_metrics["best_accuracy"])

    print(f"\nAverage Train Loss: {avg_train_loss:.4f}, Average Train Accuracy: {avg_train_accuracy:.2f}%")
    print(f"Average Validation Loss: {avg_valid_loss:.4f}, Average Validation Accuracy: {avg_valid_accuracy:.2f}%")
    print(f"Average Precision: {avg_precision:.2f}, Average Recall: {avg_recall:.2f}")
    print(f"Average Best Validation Accuracy: {avg_best_accuracy:.2f}%")

    return model, fold_metrics

In [11]:
def test_model(model, test_loader):
    model.eval() 
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc="Testing", unit="batch", total=len(test_loader)):
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass
            outputs = model(inputs)  
            preds = torch.round(torch.sigmoid(outputs))  # Binary predictions (0 or 1)

            # Adjust for correct dimensions
            all_preds.extend(preds.squeeze(1).cpu().numpy())  
            all_labels.extend(labels.cpu().numpy()) 

            correct += (preds.squeeze(1) == labels).sum().item()
            total += labels.size(0)

    accuracy = correct / total * 100
    print(f"Test Accuracy: {accuracy:.2f}%")

    return all_preds, all_labels

In [16]:
mobile = CustomMobileNet()

# Define the loss function
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(mobile.parameters(), lr=8.948291234170116e-05)

train_model(mobile ,combined_dataset, criterion, optimizer, num_epochs=10, k=3)


Fold 1/3
Using device: cuda


Epoch 1/10: 100%|██████████| 240/240 [05:00<00:00,  1.25s/batch, accuracy=83.1, loss=0.43] 


Train Loss: 0.4298, Train Accuracy: 83.14%


Validation Epoch 1/10: 100%|██████████| 120/120 [02:42<00:00,  1.36s/batch, accuracy=83.4, loss=0.41] 


Validation Accuracy: 83.36%
Precision: 0.62, Recall: 0.02


...

Epoch 10/10: 100%|██████████| 240/240 [02:05<00:00,  1.91batch/s, accuracy=96.6, loss=0.0894]


Train Loss: 0.0894, Train Accuracy: 96.56%


Validation Epoch 10/10: 100%|██████████| 120/120 [01:11<00:00,  1.69batch/s, accuracy=97.7, loss=0.0624]

Validation Accuracy: 97.67%
Precision: 0.98, Recall: 0.88

Average Train Loss: 0.2296, Average Train Accuracy: 90.72%
Average Validation Loss: 0.2159, Average Validation Accuracy: 91.58%
Average Precision: 0.82, Average Recall: 0.57
Average Best Validation Accuracy: 92.58%


(CustomMobileNet(
   (features): Sequential(
     (0): Conv2dNormActivation(
       (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
       (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (2): ReLU6(inplace=True)
     )
     (1): InvertedResidual(
       (conv): Sequential(
         (0): Conv2dNormActivation(
           (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
           (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
           (2): ReLU6(inplace=True)
         )
         (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
         (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       )
     )
     (2): InvertedResidual(
       (conv): Sequential(
         (0): Conv2dNormActivation(
           (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
         

In [17]:
mobile.unfreeze_last_layers(2)


# Define the loss function
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(mobile.parameters(), lr=8.948291234170116e-05)

train_model(mobile ,combined_dataset, criterion, optimizer, num_epochs=5, k=3)


Fold 1/3
Using device: cuda


Epoch 1/5: 100%|██████████| 240/240 [02:10<00:00,  1.84batch/s, accuracy=94.5, loss=0.143]


Train Loss: 0.1432, Train Accuracy: 94.50%


Validation Epoch 1/5: 100%|██████████| 120/120 [01:02<00:00,  1.93batch/s, accuracy=99.5, loss=0.0261]


Validation Accuracy: 99.52%
Precision: 1.00, Recall: 0.97


...

Epoch 5/5: 100%|██████████| 240/240 [02:06<00:00,  1.90batch/s, accuracy=97.6, loss=0.0629]


Train Loss: 0.0629, Train Accuracy: 97.63%


Validation Epoch 5/5: 100%|██████████| 120/120 [01:01<00:00,  1.95batch/s, accuracy=99.2, loss=0.0246]

Validation Accuracy: 99.20%
Precision: 1.00, Recall: 0.96

Average Train Loss: 0.0915, Average Train Accuracy: 96.53%
Average Validation Loss: 0.0345, Average Validation Accuracy: 98.89%
Average Precision: 0.99, Average Recall: 0.94
Average Best Validation Accuracy: 99.56%


(CustomMobileNet(
   (features): Sequential(
     (0): Conv2dNormActivation(
       (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
       (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (2): ReLU6(inplace=True)
     )
     (1): InvertedResidual(
       (conv): Sequential(
         (0): Conv2dNormActivation(
           (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
           (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
           (2): ReLU6(inplace=True)
         )
         (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
         (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       )
     )
     (2): InvertedResidual(
       (conv): Sequential(
         (0): Conv2dNormActivation(
           (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
         

In [18]:
test_model(mobile,test_loader)

Testing: 100%|██████████| 12000/12000 [01:43<00:00, 115.54batch/s]

Test Accuracy: 84.10%


([0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,